# Tonga 5m DEM creation
Load in LiDAR where it exists and combine with FABDEM to produce 5m DEMs

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pathlib

import os
import sys
module_path = os.path.abspath(os.path.join("..", "src", "pacific-dems"))
if module_path not in sys.path:
    sys.path.append(module_path)

import create_dem_functions
import islands
import create_data_paths

# Define country and resolution

In [ ]:
resolution = 5 # in meters
country_name = "tonga"

# Create paths and get contry information

In [ ]:
paths = create_data_paths.get_paths(country_name=country_name, resolution=resolution)
islands_dict = islands.get_island(country_name)

# Specify any LiDAR data

In [ ]:
# Path of LiDAR DEMs
eua_lidar = create_dem_functions.load_dem(paths["lidar"] / "Eua_1m_topobathy.tif") 
tafahi_lidar = create_dem_functions.load_dem(paths["lidar"] / "Tafahi_1m_topobathy.tif") 
vavau_lidar = create_dem_functions.load_dem(paths["lidar"] / "Vavau_1m_topobathy.tif") 

tongatapu_lidar_land = create_dem_functions.load_dem(paths["lidar"] / "TongaLidar2011_1m_dem_UTM_compressed.tif") 
tongatapu_lidar_bathy = create_dem_functions.load_dem(paths["lidar"] / "TongaLidar2011_5m_Bathymetry_UTM_compressed.tif") 
lifuka_lidar_land = create_dem_functions.load_dem(paths["lidar"] / "LifukaLidar2011_1m_DEM_UTM_compressed.tif") 
lifuka_lidar_bathy = create_dem_functions.load_dem(paths["lidar"] / "LifukaLidar2011_5m_Bathymetry_UTM_compressed.tif") 


# Add to islands
islands_dict["'eua"]["lidar"] = eua_lidar
islands_dict["niuatoputapu_tafahi_islands"]["lidar"] = tafahi_lidar
islands_dict["vava'u_group"]["lidar"] = vavau_lidar
islands_dict["tongatapu"]["lidar"] = [tongatapu_lidar_land, tongatapu_lidar_bathy]
islands_dict["ha'apai_group"]["lidar"] = [lifuka_lidar_land, lifuka_lidar_bathy]

# Get island outlines and combine FABDEMs

In [ ]:
islands_dict.pop("minerva") # remove as no FABDEM
for island_name, island_dict in islands_dict.items():
    print(f"Setup for {island_name}")
    island_dict["name"] = island_name
    island_dict["boundary"] = create_dem_functions.create_boundary_epsg4326(island_dict=island_dict, output_path=paths["output"])
    island_dict["fab"] = create_dem_functions.combine_fabs_in_boundary(island_dict=island_dict, fab_path=paths["fabdem"])

# Create DEMs over islands

In [ ]:
create_dem_functions.loop_through_islands_creating_dems(island_groups=islands_dict,
                                                        resolution=resolution, output_path=paths["output"])

In [ ]:
lidar_only_island_dict = {"tongatapu": islands_dict.pop("tongatapu")}
create_dem_functions.loop_through_islands_creating_dems_from_lidar_only(island_groups=lidar_only_island_dict,
                                                                        resolution=resolution, output_path=paths["output"],
                                                                        interpolate=False)